1. Dependencies and Imports

In [1]:
!pip install torch torchvision opacus

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from opacus import PrivacyEngine
import matplotlib.pyplot as plt


^C


ModuleNotFoundError: No module named 'torch._C'

     ---------------------------------------- 1.6/1.6 MB 9.9 MB/s eta 0:00:00
     -------------------------------------- 251.7/251.7 kB 7.8 MB/s eta 0:00:00
     -------------------------------------- 204.2/204.2 MB 3.8 MB/s eta 0:00:00
     ---------------------------------------- 6.2/6.2 MB 3.8 MB/s eta 0:00:00
     ---------------------------------------- 71.9/71.9 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
  Attempting uninstall: sympy
    Found existing installation: sympy 1.11.1
    Uninstalling sympy-1.11.1:
      Successfully uninstalled sympy-1.11.1
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1


2. Data Preparation

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('.', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('.', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000)


3. Model Definition

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, 1),
            nn.ReLU(),
            nn.Flatten(),
        )
        
        # Dynamically determine the size
        sample_input = torch.zeros(1, 1, 28, 28)
        sample_output = self.features(sample_input)
        num_features = sample_output.shape[1]

        self.classifier = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.ReLU(),
            nn.Linear(128, 10),
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


4. Define Training and Testing

In [ ]:
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    for epoch in range(1, 6):
        total_loss = 0
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch}, Loss: {total_loss/len(train_loader):.4f}")

def test(model, test_loader, device):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
    accuracy = 100. * correct / len(test_loader.dataset)
    return accuracy

5. Run Standard Training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_standard = SimpleCNN().to(device)
optimizer_standard = optim.Adam(model_standard.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

print("Training standard model...")
train(model_standard, train_loader, optimizer_standard, criterion, device)
accuracy_standard = test(model_standard, test_loader, device)
print(f"Standard Model Accuracy: {accuracy_standard:.2f}%")


6. Run Differential Privacy Training

In [ ]:
model_dp = SimpleCNN().to(device)
optimizer_dp = optim.Adam(model_dp.parameters(), lr=1e-3)
privacy_engine = PrivacyEngine()

model_dp, optimizer_dp, train_loader_dp = privacy_engine.make_private(
    module=model_dp,
    optimizer=optimizer_dp,
    data_loader=train_loader,
    noise_multiplier=1.1,  # Higher noise = more privacy
    max_grad_norm=1.0,
)

print(f"DP Model is using (ε = {privacy_engine.get_epsilon(1e-5):.2f}, δ = 1e-5)")

print("Training DP model...")
train(model_dp, train_loader_dp, optimizer_dp, criterion, device)
accuracy_dp = test(model_dp, test_loader, device)
print(f"DP Model Accuracy: {accuracy_dp:.2f}%")

epsilon = privacy_engine.get_epsilon(1e-5)


8. Compare the Results

In [ ]:
labels = ['Standard Model', 'DP Model']
accuracies = [accuracy_standard, accuracy_dp]

plt.bar(labels, accuracies, color=['blue', 'green'])
plt.ylabel('Accuracy (%)')
plt.title(f'DP ε = {epsilon:.2f}, δ = 1e-5')
plt.ylim(0, 100)
plt.show()
